In [55]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import os

import numpy as np
from tqdm.auto import tqdm
from joblib import Parallel, delayed

from audio import tools
from configs import train_config

all_wav_names = sorted(os.listdir(train_config.wav_path))
def proc(index):
    _, energy, pitch = tools.get_mel_v2_energy_pitch(
        os.path.join(train_config.wav_path, all_wav_names[index])
    )
    np.save(
        os.path.join(train_config.energy_path, 'ljspeech-energy-%05d.npy' % index),
        energy, allow_pickle=False
    )
    np.save(
        os.path.join(train_config.pitch_path, 'ljspeech-pitch-%05d.npy' % index),
        pitch, allow_pickle=False
    )

_ = Parallel(n_jobs=16)(delayed(proc)(idx) for idx in range(len(all_wav_names)))

In [131]:
import torch

i = 123

mel_gt_name = os.path.join(
    train_config.mel_ground_truth, "ljspeech-mel-%05d.npy" % (i+1))
mel_gt_target = np.load(mel_gt_name)

energy_gt_name = os.path.join(
    train_config.energy_path, "ljspeech-energy-%05d.npy" % (i))
energy_gt_target = np.load(energy_gt_name)

pitch_gt_name = os.path.join(
    train_config.pitch_path, "ljspeech-pitch-%05d.npy" % (i))
pitch_gt_target = np.load(pitch_gt_name)

mel_gt_target = torch.from_numpy(mel_gt_target)
energy_gt_target = torch.from_numpy(energy_gt_target)
pitch_gt_target = torch.from_numpy(pitch_gt_target)

mel_gt_target.shape, energy_gt_target.shape, pitch_gt_target.shape, energy_gt_target.min(), energy_gt_target.max()

(torch.Size([724, 80]),
 torch.Size([724]),
 torch.Size([724]),
 tensor(0.2604),
 tensor(133.3747))

In [132]:
from sklearn.preprocessing import StandardScaler

e_scaler, p_scaler = StandardScaler(), StandardScaler()
n_samples = len(os.listdir(train_config.energy_path))

for index in tqdm(range(n_samples)):
    energy_gt_name = os.path.join(train_config.energy_path, "ljspeech-energy-%05d.npy" % (index))
    energy_gt_target = np.load(energy_gt_name)
    pitch_gt_name = os.path.join(train_config.pitch_path, "ljspeech-pitch-%05d.npy" % (index))
    pitch_gt_target = np.load(pitch_gt_name)
    e_scaler.partial_fit(energy_gt_target.reshape((-1, 1)))
    p_scaler.partial_fit(pitch_gt_target.reshape((-1, 1)))

min_e, max_e, min_p, max_p = np.inf, -np.inf, np.inf, -np.inf
for index in tqdm(range(n_samples)):
    energy_gt_name = os.path.join(train_config.energy_path, "ljspeech-energy-%05d.npy" % (index))
    energy_gt_target = np.load(energy_gt_name)
    energy_gt_target = (energy_gt_target - e_scaler.mean_[0]) / e_scaler.scale_[0]
    # np.save(energy_gt_name, energy_gt_target)
    min_e = min(min_e, energy_gt_target.min())
    max_e = max(max_e, energy_gt_target.max())

    pitch_gt_name = os.path.join(train_config.pitch_path, "ljspeech-pitch-%05d.npy" % (index))
    pitch_gt_target = np.load(pitch_gt_name)
    pitch_gt_target = (pitch_gt_target - p_scaler.mean_[0]) / p_scaler.scale_[0]
    # np.save(pitch_gt_name, pitch_gt_target)
    min_p = min(min_p, pitch_gt_target.min())
    max_p = max(max_p, pitch_gt_target.max())
min_e, max_e, min_p, max_p

  0%|          | 0/13100 [00:00<?, ?it/s]

(0.017866513, 314.9619, 0.0, 861.0652680139365)